In [1]:
import base64
import zmq
import ray
import imageio 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy.signal import convolve2d
import cv2

In [2]:
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")

In [3]:
ray.init(address="auto")

2021-06-09 05:37:28,771	INFO worker.py:640 -- Connecting to existing Ray cluster at address: 192.168.43.232:6379


In [4]:
kernel = np.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])

In [5]:
def processImage(image): 
    image = cv2.imread(image) 
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY) 
    return image

In [6]:
@ray.remote
def convolve2D(image, kernel, padding=0, strides=1):
    # Cross Correlation
    kernel = np.flipud(np.fliplr(kernel))

    # Gather Shapes of Kernel + Image + Padding
    xKernShape = kernel.shape[0]
    yKernShape = kernel.shape[1]
    xImgShape = image.shape[0]
    yImgShape = image.shape[1]

    # Shape of Output Convolution
    xOutput = int(((xImgShape - xKernShape + 2 * padding) / strides) + 1)
    yOutput = int(((yImgShape - yKernShape + 2 * padding) / strides) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    if padding != 0:
        imagePadded = np.zeros((image.shape[0] + padding*2, image.shape[1] + padding*2))
        imagePadded[int(padding):int(-1 * padding), int(padding):int(-1 * padding)] = image
        print(imagePadded)
    else:
        imagePadded = image

    # Iterate through image
    for y in range(image.shape[1]):
        # Exit Convolution
        if y > image.shape[1] - yKernShape:
            break
        # Only Convolve if y has gone down by the specified Strides
        if y % strides == 0:
            for x in range(image.shape[0]):
                # Go to next row once kernel is out of bounds
                if x > image.shape[0] - xKernShape:
                    break
                try:
                    # Only Convolve if x has moved by the specified Strides
                    if x % strides == 0:
                        output[x, y] = (kernel * imagePadded[x: x + xKernShape, y: y + yKernShape]).sum()
                except:
                    break

    return output

In [7]:
while True:
    #  Wait for next request from client
    message = socket.recv()
    print("Received request"
    ba = bytearray(base64.b64decode(message))
    f = open("./temp/server_received.png", 'wb')
    f.write(ba)
    f.close()

    print("processing ... ")
    image = processImage("./temp/server_received.png")
    object_id = convolve2D.remote(image,kernel)
    #print(object_id)
    data=ray.get(object_id)
    cv2.imwrite('./temp/temp.png', data)
    
    
    print("sending back results")
    f = open('./temp/temp.png','rb')
    bytes = bytearray(f.read())
    strng = base64.b64encode(bytes)
    socket.send(strng)
    f.close()
    
ray.shutdown()

ValueError: too many values to unpack (expected 2)